# Aplikasi Tracker Lamaran Pekerjaan


Langkah 1: Persiapan Awal (Konfigurasi & Setup Database)

Tujuan: Menyiapkan file konfigurasi dasar dan sebuah skrip untuk membuat database SQLite beserta tabelnya untuk pertama kali. Langkah ini adalah fondasi untuk seluruh aplikasi.

File 1: konfigurasi.py

In [ ]:
# konfigurasi.py
import os

# Direktori dasar proyek
BASE_DIR = os.path.dirname(os.path.abspath(__file__))

# Nama file database
NAMA_DB = 'lamaran_pekerjaan.db'

# Path lengkap ke file database
DB_PATH = os.path.join(BASE_DIR, NAMA_DB)

# Kategori Status Lamaran
KATEGORI_STATUS = ["Rencana Dilamar", "Sudah Apply", "Ditolak", "Diterima"]

File 2: setup_db.py 

Program ini hanya perlu dijalankan sekali dari terminal (python setup_db.py) untuk membuat file database dan struktur tabelnya. Setelah perintah python setup_db.py dijalankan, file dengan nama lamaran_pekerjaan.db akan otomatis dibuat di dalam direktori proyek. File ini berisi table pekerjaan yang kosong dan siap untuk diisi data.

In [1]:
# setup_db.py
import sqlite3
from konfigurasi import DB_PATH

def setup_database():
    """Membuat database dan tabel pekerjaan dengan semua kolom yang diperlukan."""
    print(f"Membuat database di: {DB_PATH}")
    try:
        with sqlite3.connect(DB_PATH) as conn:
            cursor = conn.cursor()
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS pekerjaan (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    nama_perusahaan TEXT NOT NULL,
                    latitude REAL NOT NULL,
                    longitude REAL NOT NULL,
                    status TEXT NOT NULL,
                    tanggal_lamar DATE NOT NULL,
                    catatan TEXT
                );
            """)
            print("Tabel 'pekerjaan' berhasil dibuat atau sudah ada.")
    except sqlite3.Error as e:
        print(f"Error saat membuat database: {e}")

if __name__ == "__main__":
    print("--- Memulai Setup Database Aplikasi ---")
    setup_database()
    print("--- Setup Database Selesai ---")


--- Memulai Setup Database Aplikasi ---
Membuat database di: d:\TUGAS\Semester 2\PBO\tubes\lamaran_pekerjaan.db
Tabel 'pekerjaan' berhasil dibuat atau sudah ada.
--- Setup Database Selesai ---


Langkah 3: Modul Model Data (model.py)

Tujuan: Mendefinisikan kelas Pekerjaan yang merepresentasikan struktur data untuk satu entitas lamaran pekerjaan. 
File: model.py

In [3]:
# model.py
import datetime
from konfigurasi import KATEGORI_STATUS

class Pekerjaan:
    """Merepresentasikan satu entitas lamaran pekerjaan."""
    def __init__(self, nama_perusahaan, latitude, longitude, status, tanggal_lamar, catatan="", id_pekerjaan=None):
        if not nama_perusahaan:
            raise ValueError("Nama perusahaan tidak boleh kosong.")
        if status not in KATEGORI_STATUS:
            raise ValueError(f"Status tidak valid. Pilih dari: {KATEGORI_STATUS}")
        if not isinstance(tanggal_lamar, datetime.date):
            raise TypeError("Format tanggal tidak valid.")

        self.id = id_pekerjaan
        self.nama_perusahaan = str(nama_perusahaan)
        self.status = status
        self.tanggal_lamar = tanggal_lamar
        self.catatan = str(catatan)
        try:
            self.latitude = float(latitude)
            self.longitude = float(longitude)
        except (ValueError, TypeError):
            raise ValueError("Latitude dan Longitude harus berupa angka yang valid.")

    def __repr__(self):
        return (f"Pekerjaan(ID: {self.id}, Perusahaan: '{self.nama_perusahaan}', "
                f"Status: {self.status}, Tanggal: {self.tanggal_lamar})")


Langkah 4: Modul Manajer Logika Bisnis (manajer_pekerjaan.py)
Tujuan: Membuat kelas ManajerPekerjaan yang berisi semua logika bisnis aplikasi. Kelas ini menjadi perantara antara antarmuka pengguna dan database. 
File: manajer_pekerjaan.py

In [5]:
# manajer_pekerjaan.py
import pandas as pd
import database
from model import Pekerjaan

class ManajerPekerjaan:
    """Mengelola logika bisnis untuk lamaran pekerjaan."""
    def tambah_pekerjaan(self, pekerjaan: Pekerjaan):
        """Menyimpan data pekerjaan baru ke database."""
        sql = "INSERT INTO pekerjaan (nama_perusahaan, latitude, longitude, status, tanggal_lamar, catatan) VALUES (?, ?, ?, ?, ?, ?)"
        tanggal_str = pekerjaan.tanggal_lamar.strftime('%Y-%m-%d')
        params = (pekerjaan.nama_perusahaan, pekerjaan.latitude, pekerjaan.longitude, pekerjaan.status, tanggal_str, pekerjaan.catatan)
        last_id = database.execute_query(sql, params)
        if last_id is not None:
            pekerjaan.id = last_id
            return True
        return False

    def get_semua_pekerjaan(self):
        """Mengambil semua data pekerjaan dari database."""
        query = "SELECT id, nama_perusahaan, tanggal_lamar as 'Tanggal Lamar', status, catatan, latitude, longitude FROM pekerjaan ORDER BY tanggal_lamar DESC, id DESC"
        df = database.get_dataframe(query)
        return df
    
    def get_pekerjaan_by_id(self, id_pekerjaan: int):
        """Mengambil satu data pekerjaan berdasarkan ID."""
        query = "SELECT * FROM pekerjaan WHERE id = ?"
        params = (id_pekerjaan,)
        
        df = database.get_dataframe(query, params=params)

        if not df.empty:
            row = df.iloc[0]
            
            try:
                import datetime
                # Format di DB adalah YYYY-MM-DD
                tgl_lamar_obj = datetime.datetime.strptime(row['tanggal_lamar'], '%Y-%m-%d').date()
            except (ValueError, TypeError):
                tgl_lamar_obj = datetime.date.today() # Fallback jika format salah

            return Pekerjaan(
                id_pekerjaan=int(row['id']),
                nama_perusahaan=row['nama_perusahaan'],
                latitude=row['latitude'],
                longitude=row['longitude'],
                status=row['status'],
                tanggal_lamar=tgl_lamar_obj,
                catatan=row['catatan']
            )
        return None
        
    def hapus_pekerjaan(self, id_pekerjaan: int):
        """Menghapus data pekerjaan berdasarkan ID."""
        sql = "DELETE FROM pekerjaan WHERE id = ?"
        params = (id_pekerjaan,)
        try:
            database.execute_query(sql, params)
            return True
        except Exception as e:
            print(f"Error saat menghapus: {e}")
            return False
    
    def update_pekerjaan(self, id_pekerjaan: int, pekerjaan: Pekerjaan):
        """Memperbarui semua data lamaran pekerjaan berdasarkan ID."""
        sql = """UPDATE pekerjaan SET 
                    nama_perusahaan = ?,
                    tanggal_lamar = ?,
                    status = ?,
                    latitude = ?,
                    longitude = ?,
                    catatan = ?
                 WHERE id = ?"""
        params = (
            pekerjaan.nama_perusahaan,
            pekerjaan.tanggal_lamar.strftime('%Y-%m-%d'),
            pekerjaan.status,
            pekerjaan.latitude,
            pekerjaan.longitude,
            pekerjaan.catatan,
            id_pekerjaan
        )
        try:
            database.execute_query(sql, params)
            return True
        except Exception as e:
            print(f"Error saat memperbarui data: {e}")
            return False
            
    def get_ringkasan_status(self):
        """Menghitung jumlah pekerjaan untuk setiap status."""
        query = "SELECT status, COUNT(*) as jumlah FROM pekerjaan GROUP BY status"
        df = database.get_dataframe(query)
        if df.empty:
            return {}
        return pd.Series(df.jumlah.values, index=df.status).to_dict()


Langkah 5: Aplikasi Utama Streamlit (app.py)

Tujuan: Membangun antarmuka pengguna (UI) web yang interaktif menggunakan Streamlit, yang mengintegrasikan semua komponen backend yang telah dibuat. 

File: app.py

In [8]:
# app.py
import streamlit as st
import pandas as pd
import folium
import datetime
from streamlit_folium import st_folium
from manajer_pekerjaan import ManajerPekerjaan
from model import Pekerjaan
from konfigurasi import KATEGORI_STATUS

# --- Konfigurasi Halaman & Inisialisasi ---
st.set_page_config(page_title="Pelacak Lamaran", layout="wide")
manajer = ManajerPekerjaan()

# Pemetaan status ke warna marker untuk peta
WARNA_MARKER = { "Rencana Dilamar": "blue", "Sudah Apply": "orange", "Ditolak": "red", "Diterima": "green" }

# --- Sidebar untuk Input & Kontrol ---
with st.sidebar:
    daftar_pekerjaan_sidebar = manajer.get_semua_pekerjaan()

    st.header("📌 Tambah Lamaran Baru")
    with st.form("form_tambah", clear_on_submit=True):
        nama_perusahaan = st.text_input("Nama Perusahaan*", placeholder="Contoh: PT Herco Digital")
        tanggal_lamar = st.date_input("Tanggal Melamar*", value=datetime.date.today())
        status = st.selectbox("Status Awal*", KATEGORI_STATUS)
        with st.expander("Data Opsional"):
            latitude = st.text_input("Latitude", placeholder="-6.984369")
            longitude = st.text_input("Longitude", placeholder="110.431240")
            catatan = st.text_area("Catatan", placeholder="Contoh: Perlu siapkan CV dan portofolio data scientist.")
        
        if st.form_submit_button("Simpan Lamaran", use_container_width=True):
            lat_val = latitude if latitude else "0"
            lon_val = longitude if longitude else "0"
            try:
                pekerjaan_baru = Pekerjaan(nama_perusahaan, lat_val, lon_val, status, tanggal_lamar, catatan)
                if manajer.tambah_pekerjaan(pekerjaan_baru):
                    st.success("Lamaran berhasil disimpan!")
            except (ValueError, TypeError) as e:
                st.error(f"Error: {e}")

    st.divider()

    # --- Edit Data ---
    st.header("✏️ Edit Lamaran")
    if daftar_pekerjaan_sidebar.empty:
        st.info("Tidak ada data untuk diubah.")
    else:
        pilihan_lamaran_edit = [f"{row['id']} - {row['nama_perusahaan']}" for _, row in daftar_pekerjaan_sidebar.iterrows()]
        id_edit_pilihan = st.selectbox("Pilih lamaran untuk diedit", pilihan_lamaran_edit, index=None, placeholder="Pilih lamaran...")

        if id_edit_pilihan:
            id_to_edit = int(id_edit_pilihan.split(" - ")[0])
            data_lama = manajer.get_pekerjaan_by_id(id_to_edit)

            if data_lama:
                with st.form("form_edit"):
                    st.subheader(f"Mengedit: {data_lama.nama_perusahaan}")
                    edit_nama = st.text_input("Nama Perusahaan", value=data_lama.nama_perusahaan)
                    edit_tanggal = st.date_input("Tanggal Lamar", value=data_lama.tanggal_lamar)
                    edit_status = st.selectbox("Status Lamaran", KATEGORI_STATUS, index=KATEGORI_STATUS.index(data_lama.status))
                    edit_lat = st.text_input("Latitude", value=str(data_lama.latitude))
                    edit_lon = st.text_input("Longitude", value=str(data_lama.longitude))
                    edit_catatan = st.text_area("Catatan", value=data_lama.catatan)

                    if st.form_submit_button("Simpan Perubahan", use_container_width=True, type="primary"):
                        try:
                            pekerjaan_diperbarui = Pekerjaan(edit_nama, edit_lat, edit_lon, edit_status, edit_tanggal, edit_catatan)
                            if manajer.update_pekerjaan(id_to_edit, pekerjaan_diperbarui):
                                st.success("Data berhasil diperbarui!")
                                st.rerun()
                            else:
                                st.error("Gagal memperbarui data.")
                        except (ValueError, TypeError) as e:
                            st.error(f"Error: {e}")
    st.divider()

    # --- Hapus Data ---
    st.header("🗑️ Hapus Lamaran")
    if daftar_pekerjaan_sidebar.empty:
        st.info("Tidak ada data untuk dihapus.")
    else:
        pilihan_lamaran_hapus = [f"{row['id']} - {row['nama_perusahaan']}" for _, row in daftar_pekerjaan_sidebar.iterrows()]
        id_hapus_pilihan = st.selectbox("Pilih lamaran untuk dihapus", pilihan_lamaran_hapus, index=None, placeholder="Pilih lamaran...")
        
        if id_hapus_pilihan:
            id_to_delete = int(id_hapus_pilihan.split(" - ")[0])
            if st.button("Hapus Lamaran Terpilih", use_container_width=True, type="secondary"):
                st.session_state.id_confirm_delete = id_to_delete

# --- Tampilan Utama dengan Tabs ---
st.title("🗂️ Pelacak Lamaran Pekerjaan")

tab1, tab2, tab3 = st.tabs(["📊 Papan Status", "📜 Riwayat Lengkap", "🗺️ Peta Lamaran"])

with tab1:
    st.header("Ringkasan Status Lamaran")
    ringkasan = manajer.get_ringkasan_status()
    total_rencana = ringkasan.get("Rencana Dilamar", 0)
    total_apply = ringkasan.get("Sudah Apply", 0)
    total_diterima = ringkasan.get("Diterima", 0)
    total_ditolak = ringkasan.get("Ditolak", 0)

    col_met1, col_met2, col_met3, col_met4 = st.columns(4)
    col_met1.metric("Rencana Dilamar 📝", total_rencana)
    col_met2.metric("Sudah Apply 📩", total_apply)
    col_met3.metric("Diterima 🎉", total_diterima)
    col_met4.metric("Ditolak 😔", total_ditolak)
    
    # Logika konfirmasi penghapusan
    if 'id_confirm_delete' in st.session_state and st.session_state.id_confirm_delete:
        id_delete = st.session_state.id_confirm_delete
        nama_perusahaan_delete = manajer.get_pekerjaan_by_id(id_delete).nama_perusahaan
        st.warning(f"Anda yakin ingin menghapus lamaran untuk **{nama_perusahaan_delete}** (ID: {id_delete})?")
        col_confirm, col_cancel = st.columns(2)
        if col_confirm.button("Ya, Konfirmasi Hapus", type="primary", use_container_width=True):
            if manajer.hapus_pekerjaan(id_delete):
                del st.session_state.id_confirm_delete
                st.rerun()
        if col_cancel.button("Batal", use_container_width=True):
            del st.session_state.id_confirm_delete
            st.rerun()

with tab2:
    st.header("Riwayat Lengkap Lamaran")
    daftar_pekerjaan = manajer.get_semua_pekerjaan()
    if daftar_pekerjaan.empty:
        st.info("Belum ada data lamaran yang tercatat.")
    else:
        st.dataframe(daftar_pekerjaan.drop(columns=['latitude', 'longitude']), use_container_width=True, hide_index=True)

with tab3:
    st.header("Peta Sebaran Lokasi Lamaran")
    peta_pekerjaan = manajer.get_semua_pekerjaan()
    peta_pekerjaan_valid = peta_pekerjaan[(peta_pekerjaan['latitude'] != 0.0) & (peta_pekerjaan['longitude'] != 0.0)]
    
    # --- Fitur Pencarian Peta ---
    nama_perusahaan_list = ["Semua Lokasi"] + peta_pekerjaan_valid['nama_perusahaan'].tolist()
    cari_perusahaan = st.selectbox("Cari Alamat Lamaran", nama_perusahaan_list)
    
    # Tentukan pusat peta
    map_center = [-6.9175, 107.6191] # Default bujur peta adalah Bandung
    zoom_level = 9
    
    if cari_perusahaan != "Semua Lokasi":
        lokasi_terpilih = peta_pekerjaan_valid[peta_pekerjaan_valid['nama_perusahaan'] == cari_perusahaan].iloc[0]
        map_center = [lokasi_terpilih['latitude'], lokasi_terpilih['longitude']]
        zoom_level = 15 # Zoom lebih dekat saat mencari
    elif not peta_pekerjaan_valid.empty:
        map_center = [peta_pekerjaan_valid.iloc[0]['latitude'], peta_pekerjaan_valid.iloc[0]['longitude']]
        zoom_level = 11

    # Buat Peta Folium
    m = folium.Map(location=map_center, zoom_start=zoom_level)

    # Tambahkan marker ke peta
    if not peta_pekerjaan_valid.empty:
        for _, row in peta_pekerjaan_valid.iterrows():
            warna_ikon = WARNA_MARKER.get(row['status'], 'gray')
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=f"<b>{row['nama_perusahaan']}</b><br>Status: {row['status']}",
                tooltip=f"{row['nama_perusahaan']} ({row['status']})",
                icon=folium.Icon(color=warna_ikon, icon='briefcase', prefix='fa')
            ).add_to(m)
    
    st_folium(m, use_container_width=True, height=600)


2025-07-03 13:28:22.861 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 13:28:22.865 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 13:28:22.881 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 13:28:23.042 
  command:

    streamlit run d:\TUGAS\Semester 2\PBO\tubes\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-07-03 13:28:23.044 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 13:28:23.046 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-03 13:28:23.047 Thread '

Langkah 6: Menjalankan dan Menguji Aplikasi
Tujuan: Memastikan semua modul yang telah dibuat dapat bekerja sama dengan baik dan aplikasi berfungsi sesuai dengan yang diharapkan. 
Langkah-langkah:
1.	Pastikan semua file (konfigurasi.py, database.py, model.py, manajer_pekerjaan.py, app.py) berada dalam satu direktori kerja.
2.	Jika belum, jalankan skrip setup_db.py sekali saja untuk membuat file database: python setup_db.py. 
3.	Buka terminal pada direktori tersebut dan jalankan aplikasi utama: streamlit run app.py. 
4.	Sebuah tab baru pada browser akan terbuka secara otomatis, menampilkan antarmuka aplikasi.
5.	Lakukan pengujian menyeluruh pada semua fungsionalitas:
o	Tambah Data: Isi form di sidebar dan simpan lamaran baru. Periksa apakah data muncul di tabel riwayat dan ringkasan diperbarui.
o	Edit Data: Pilih salah satu lamaran dari dropdown "Edit Lamaran", ubah beberapa datanya, lalu simpan. Pastikan perubahan tecermin di tabel riwayat.
o	Hapus Data: Pilih lamaran dari dropdown "Hapus Lamaran", lalu konfirmasi penghapusan. Pastikan data tersebut hilang dari semua tampilan.
o	Navigasi Tab: Pindah antar tab "Papan Status", "Riwayat Lengkap", dan "Peta Lamaran" untuk memastikan semuanya tampil dengan benar.
o	Fitur Peta: Gunakan dropdown "Cari Alamat" di tab peta untuk memusatkan peta pada lokasi tertentu. Klik pada marker untuk melihat detail popup.